In [27]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
np.set_printoptions(suppress = True, precision= 6)

In [28]:
df1 = pd.read_csv("Plant_1_Generation_Data.csv")
df2 = pd.read_csv("Plant_1_Weather_Sensor_Data.csv")

In [29]:
df1['DATE_TIME'] = df1['DATE_TIME'].astype('datetime64[ns]')
df2['DATE_TIME'] = df2['DATE_TIME'].astype('datetime64[ns]').replace('/','-')

df = pd.merge(df1,df2, how='left', on='DATE_TIME')

In [30]:
df = df.drop(columns=['PLANT_ID_x', 'SOURCE_KEY_x','PLANT_ID_y', 'SOURCE_KEY_y','DAILY_YIELD','TOTAL_YIELD', 'AC_POWER'])

df.head()

,DATE_TIME,DC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15,0.0,25.184316,22.857507,0.0
1,2020-05-15,0.0,25.184316,22.857507,0.0
2,2020-05-15,0.0,25.184316,22.857507,0.0
3,2020-05-15,0.0,25.184316,22.857507,0.0
4,2020-05-15,0.0,25.184316,22.857507,0.0


In [31]:
def new_cols(x):
    x = x[x['IRRADIATION'] != 0.0]
        
    # Filling data with farword filling
    x = x.fillna(method = 'ffill')
    
    x['DATE_TIME'] = pd.to_datetime(x['DATE_TIME'])
    
    daily_data = x.groupby(x['DATE_TIME'].dt.date)['DC_POWER'].mean()
    x['DAY_DC_POWER'] = x['DATE_TIME'].dt.date.map(daily_data)
    
    weekly_sum = daily_data.rolling(window=7).sum()
    x['WEEK_DC_POWER'] = x['DATE_TIME'].dt.date.map(weekly_sum)
    
    x['WEEK_DC_POWER'].fillna(0, inplace= True)
    
    return x

In [32]:
df = new_cols(df)
df.head()

,DATE_TIME,DC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DAY_DC_POWER,WEEK_DC_POWER
489,2020-05-15 05:45:00,0.0,24.289211,23.096692,0.000863,4813.720396,0.0
490,2020-05-15 05:45:00,0.0,24.289211,23.096692,0.000863,4813.720396,0.0
491,2020-05-15 05:45:00,0.0,24.289211,23.096692,0.000863,4813.720396,0.0
492,2020-05-15 05:45:00,0.0,24.289211,23.096692,0.000863,4813.720396,0.0
493,2020-05-15 05:45:00,0.0,24.289211,23.096692,0.000863,4813.720396,0.0


In [33]:
from sklearn.base import BaseEstimator, TransformerMixin

In [34]:
class Preprocessing(BaseEstimator, TransformerMixin):
    def fit(self,x, y=None):
        return self
    
    def transform(self,x):
        
        # Filling data with farword filling
        x = x.fillna(method = 'ffill')
        x = x.copy()
        # Feature extraction
        x['DATE_TIME'] = pd.to_datetime(x['DATE_TIME'])
        
        x['Month'] = x['DATE_TIME'].dt.month
        x['Day'] = x['DATE_TIME'].dt.day
        x['Day_of_week'] = x['DATE_TIME'].dt.weekday

        x['Hour'] = x['DATE_TIME'].dt.hour
        x['Minute'] = x['DATE_TIME'].dt.minute

        
        
        return x.drop(columns= ['DATE_TIME'])
            

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [36]:
class Encoding(BaseEstimator, TransformerMixin):
    def fit(self,x, y=None):
        return self
    
    def transform (self, x):
        x_encoded = x.copy()
        
        # Time encoding
        x_encoded['hour_sin'] = np.sin(2 * np.pi * x_encoded['Hour'] / 24)
        x_encoded['hour_cos'] = np.cos(2 * np.pi * x_encoded['Hour'] / 24)

        x_encoded['minute_sin'] = np.sin(2 * np.pi * x_encoded['Minute'] / 60)
        x_encoded['minute_cos'] = np.cos(2 * np.pi * x_encoded['Minute'] / 60)
        
        # Date encoding
        x_encoded['month_sin'] = np.sin(2 * np.pi * (x_encoded['Month'] -1) / 12)
        x_encoded['month_cos'] = np.cos(2 * np.pi * (x_encoded['Month'] -1) / 12)

        x_encoded['day_sin'] = np.sin(2 * np.pi * x_encoded['Day']- 1/ 31 )
        x_encoded['day_cos'] = np.cos(2 * np.pi * x_encoded['Day'] -1/ 31 )

        x_encoded['week_day_sin'] = np.sin(2 * np.pi * x_encoded['Day_of_week'] / 7)
        x_encoded['week_day_cos'] = np.cos(2 * np.pi * x_encoded['Day_of_week'] / 7)
        
        drop_cols = ['Hour', 'Minute','Month', 'Day', 'Day_of_week' ]
        
        
        return x_encoded.drop(columns= drop_cols)

In [37]:
df.columns

Index(['DATE_TIME', 'DC_POWER', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'DAY_DC_POWER', 'WEEK_DC_POWER'],
      dtype='object')

In [52]:
x= df.drop(columns= ['DC_POWER', 'DAY_DC_POWER', 'WEEK_DC_POWER'])
y = df[['DC_POWER', 'DAY_DC_POWER']]

In [53]:
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FunctionTransformer

In [55]:
pipe = Pipeline([
    ('Preprocessing', Preprocessing()),
    ('Encoding', Encoding()),
    ('Scale', StandardScaler()),
    ('Clf',RandomForestRegressor(n_estimators=100))
])

In [56]:
pipe.fit(x_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('Preprocessing', ...), ('Encoding', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_sam

In [57]:
y_pred = pipe.predict(x_test)
y_pred 

array([[ 7191.510267,  5908.736493],
       [    0.      ,  3567.754122],
       [ 8115.273056,  3536.986299],
       ...,
       [   35.726313,  6977.884375],
       [10256.154176,  5097.25245 ],
       [ 8741.356391,  2472.70545 ]], shape=(16101, 2))

In [58]:
y_test

,DC_POWER,DAY_DC_POWER
5249,7850.000,5908.736493
39583,0.000,3567.754122
48903,7489.125,3536.986299
44738,8174.000,5116.909375
20016,8834.125,6977.884375
...,...,...
40389,10675.125,3567.754122
41609,0.000,3316.573958
20822,27.000,6977.884375
63633,9512.750,5097.252450


In [66]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
r2 = r2_score(y_pred=y_pred, y_true= y_test)
mae = mean_absolute_error(y_pred=y_pred, y_true= y_test)
mse = mean_squared_error(y_pred=y_pred, y_true= y_test)
adj_r2 = 1-((1 - r2) * (len(y_test)-1)/(len(y_test)-13-1))
print(r2,adj_r2, mse, mae)

0.9891666851259734 0.9891579306600468 187694.63644687165 122.8763770210472


In [60]:
test_p = Preprocessing()
test_e = Encoding()
scale = StandardScaler()
new_x_train = test_e.fit_transform(test_p.fit_transform(x_train))
scaled_x_train = scale.fit_transform(new_x_train)
clf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100))
clf.fit(scaled_x_train,y_train)

,estimator estimator: estimator objectAn estimator object implementing :term:`fit` and :term:`predict`.,RandomForestRegressor()
,"n_jobs n_jobs: int or None, optional (default=None)The number of jobs to run in parallel.:meth:`fit`, :meth:`predict` and :meth:`partial_fit` (if supportedby the passed estimator) will be parallelized for each target.When individual estimators are fast to train or predict,using ``n_jobs > 1`` can result in slower performance dueto the parallelism overhead.``None`` means `1` unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all available processes / threads.See :term:`Glossary ` for more details... versionchanged:: 0.20 `n_jobs` default changed from `1` to `None`.",None
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least oneval

In [65]:
new_x_train

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour_sin,hour_cos,minute_sin,minute_cos,month_sin,month_cos,day_sin,day_cos,week_day_sin,week_day_cos
7353,27.225153,42.824544,0.555787,-7.071068e-01,-7.071068e-01,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
53622,23.801898,23.067796,0.002115,-9.659258e-01,2.588190e-01,1.000000e+00,2.832769e-16,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
18487,33.761304,54.006864,0.476600,-7.071068e-01,-7.071068e-01,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,-0.781831,0.623490
30511,28.536003,27.203674,0.019436,-1.000000e+00,-1.836970e-16,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,-0.974928,-0.222521
24439,31.947034,62.420866,1.036561,1.224647e-16,-1.000000e+00,0.000000e+00,1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20203,31.036269,60.584949,0.947985,2.588190e-01,-9.659258e-01,5.665539e-16,-1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
61442,25.218286,37.815925,0.492024,1.224647e-16,-1.000000e+00,0.000000e+00,1.000000e+00,0.500000,-8.660254e-01,-0.032252,0.99948,-0.781831,0.623490
53416,23.801898,23.067796,0.002115,-8.660254e-01,-5.000000e-01,-1.000000e+00,-1.836970e-16,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
1349,34.402852,46.744281,0.565196,-7.071068e-01,-7.071068e-01,5.665539e-16,-1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,-0.433884,-0.900969


In [61]:
new_x_train

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,hour_sin,hour_cos,minute_sin,minute_cos,month_sin,month_cos,day_sin,day_cos,week_day_sin,week_day_cos
7353,27.225153,42.824544,0.555787,-7.071068e-01,-7.071068e-01,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
53622,23.801898,23.067796,0.002115,-9.659258e-01,2.588190e-01,1.000000e+00,2.832769e-16,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
18487,33.761304,54.006864,0.476600,-7.071068e-01,-7.071068e-01,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,-0.781831,0.623490
30511,28.536003,27.203674,0.019436,-1.000000e+00,-1.836970e-16,1.000000e+00,2.832769e-16,0.866025,-5.000000e-01,-0.032252,0.99948,-0.974928,-0.222521
24439,31.947034,62.420866,1.036561,1.224647e-16,-1.000000e+00,0.000000e+00,1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20203,31.036269,60.584949,0.947985,2.588190e-01,-9.659258e-01,5.665539e-16,-1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,0.000000,1.000000
61442,25.218286,37.815925,0.492024,1.224647e-16,-1.000000e+00,0.000000e+00,1.000000e+00,0.500000,-8.660254e-01,-0.032252,0.99948,-0.781831,0.623490
53416,23.801898,23.067796,0.002115,-8.660254e-01,-5.000000e-01,-1.000000e+00,-1.836970e-16,-1.000000,-1.836970e-16,-0.032252,0.99948,0.781831,0.623490
1349,34.402852,46.744281,0.565196,-7.071068e-01,-7.071068e-01,5.665539e-16,-1.000000e+00,0.866025,-5.000000e-01,-0.032252,0.99948,-0.433884,-0.900969


In [62]:
new_x_test = test_e.fit_transform(test_p.fit_transform(x_test))
scaled_x_test = scale.fit_transform(new_x_test)
clf.predict(scaled_x_test)

array([[ 7193.223969,  5908.736493],
       [    0.      ,  5057.011884],
       [ 8099.070463,  5033.643584],
       ...,
       [   29.366371,  6705.082918],
       [10220.099501,  5097.25245 ],
       [ 8738.927131,  5741.938244]], shape=(16101, 2))

In [63]:
x_train

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
7353,2020-05-18 15:15:00,27.225153,42.824544,0.555787
53622,2020-10-06 19:15:00,23.801898,23.067796,0.002115
18487,2020-05-24 15:15:00,33.761304,54.006864,0.476600
30511,2020-05-30 18:15:00,28.536003,27.203674,0.019436
24439,2020-05-27 12:00:00,31.947034,62.420866,1.036561
...,...,...,...,...
20203,2020-05-25 11:30:00,31.036269,60.584949,0.947985
61442,2020-06-14 12:00:00,25.218286,37.815925,0.492024
53416,2020-10-06 16:45:00,23.801898,23.067796,0.002115
1349,2020-05-15 15:30:00,34.402852,46.744281,0.565196
